<a href="https://colab.research.google.com/github/chlochloguo/SQLexploreproject/blob/main/SQL_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Import libraries
import pandas as pd

# Load Excel file
file_path = '/content/drive/MyDrive/Resume Projects/SQL Tableau project /CovidDeaths (3).xlsx'
CovidDeaths = pd.read_excel(file_path)
file_paths = '/content/drive/MyDrive/Resume Projects/SQL Tableau project /CovidVaccinations.xlsx'
CovidVaccinations=pd.read_excel(file_paths)
# Load SQL extension
%load_ext sql
%sql sqlite://

# Drop old table if it exists
%sql DROP TABLE IF EXISTS CovidDeaths;
%sql DROP TABLE IF EXISTS CovidVaccinations;
# Persist the DataFrame as a SQL table
%sql --persist CovidDeaths
%sql --persist CovidVaccinations

In [ ]:
%pip install --upgrade prettytable ipython-sql
%config SqlMagic.style = '_DEPRECATED_DEFAULT'

In [ ]:
%%sql
SELECT location, date, total_cases, new_cases, total_deaths, population
From CovidDeaths
ORDER BY Location, date


In [ ]:
## looking at total cases vs total deaths
## shows likelihood of dying if you contract covid in your country
%%sql
SELECT location, date, total_cases, total_deaths, (total_deaths/total_cases)*100 as DeathPercentage
From CovidDeaths
Where location like '%states%'
ORDER BY Location, date

In [ ]:
## looking at total cases vs population
## shows what percentage of population got covid
%%sql
SELECT location, date, population, total_cases, (total_cases/population)*100 as PercentPopulationInfected
From CovidDeaths
ORDER BY Location, date

In [ ]:
## looking at countries with hightest infection rate compared to population
%%sql
SELECT location, population,Max(total_cases) as HighestInfectionCOunt,
 MAX(total_cases/population)*100 as PercentPopulationInfected
From CovidDeaths
Group by Location, population
ORDER BY PercentPopulationInfected desc

In [ ]:
# Showing ccountries with highest death count per population
%%sql
SELECT location, Max(total_deaths) as TotalDeathCount
From CovidDeaths
Where continent is not null
Group by Location
ORDER BY TotalDeathCount desc

In [ ]:
## Showing continents with the highest death count
%%sql
SELECT continent, MAX(total_deaths) as TotalDeathCount
From CovidDeaths
Where continent is not null
Group by continent
ORDER BY TotalDeathCount desc

In [ ]:
## global numbers
%%sql
Select SUM(new_cases) as total_cases, SUM(cast(new_deaths as int)) as total_deaths,
        SUM(cast(new_deaths as int))/SUM(new_cases)*100 as DeathPercentage
From CovidDeaths
-- location like '%state%'
Where continent is not null
ORDER by SUM(new_cases),
 SUM(cast(new_deaths as int))


In [ ]:
# Looking at total population vs accinations
%%sql
SELECT
    dea.continent,
    dea.location,
    dea.date,
    dea.population,
    vac.new_vaccinations,
    SUM(CAST(vac.new_vaccinations AS BIGINT)) OVER (
        PARTITION BY dea.location
        ORDER BY dea.location, dea.date
    ) AS RollingPeopleVaccinated,
    (SUM(CAST(vac.new_vaccinations AS BIGINT)) OVER (
        PARTITION BY dea.location
        ORDER BY dea.location, dea.date
    ) * 1.0 / dea.population) * 100 AS PercentVaccinated
FROM CovidDeaths AS dea
JOIN CovidVaccinations AS vac
    ON dea.location = vac.location
   AND dea.date = vac.date
WHERE dea.continent IS NOT NULL
ORDER BY dea.location, dea.date;


In [ ]:
# CTE
%%sql
WITH PopvsVac (
    Continent,
    Location,
    Date,
    Population,
    New_Vaccinations,
    RollingPeopleVaccinated,
    PercentVaccinated
) AS (
    SELECT
        dea.continent,
        dea.location,
        dea.date,
        dea.population,
        vac.new_vaccinations,
        SUM(CAST(vac.new_vaccinations AS BIGINT)) OVER (
            PARTITION BY dea.location
            ORDER BY dea.location, dea.date
        ) AS RollingPeopleVaccinated,
        (SUM(CAST(vac.new_vaccinations AS BIGINT)) OVER (
            PARTITION BY dea.location
            ORDER BY dea.location, dea.date
        ) * 1.0 / dea.population) * 100 AS PercentVaccinated
    FROM CovidDeaths AS dea
    JOIN CovidVaccinations AS vac
        ON dea.location = vac.location
       AND dea.date = vac.date
    WHERE dea.continent IS NOT NULL
)
SELECT *,(RollingPeopleVaccinated/Population)*100
FROM PopvsVac


In [ ]:
# Temp Table
%%sql
CREATE TABLE IF NOT EXISTS PercentPopulationVaccinated (
    Continent NVARCHAR(255),
    Location NVARCHAR(255),
    Date DATETIME,
    Population NUMERIC,
    New_vaccinations NUMERIC,
    RollingPeopleVaccinated NUMERIC
);

INSERT INTO PercentPopulationVaccinated
SELECT
    dea.continent,
    dea.location,
    dea.date,
    dea.population,
    vac.new_vaccinations,
    SUM(CAST(vac.new_vaccinations AS BIGINT)) OVER (
            PARTITION BY dea.location
            ORDER BY dea.location, dea.date
        ) AS RollingPeopleVaccinated
FROM CovidDeaths AS dea
JOIN CovidVaccinations AS vac
    ON dea.location = vac.location
   AND dea.date = vac.date
WHERE dea.continent IS NOT NULL;

SELECT *,(RollingPeopleVaccinated/Population)*100
From PercentPopulationVaccinated



In [ ]:
# Cretae View to store data for later
%%sql
DROP VIEW IF EXISTS PercentPopulationVaccinated;
CREATE VIEW PercentPopulationVaccinated AS
SELECT
    dea.continent,
    dea.location,
    dea.date,
    dea.population,
    vac.new_vaccinations,
    SUM(CAST(vac.new_vaccinations AS INTEGER)) OVER (
            PARTITION BY dea.location
            ORDER BY dea.location, dea.date ) AS RollingPeopleVaccinated
FROM CovidDeaths AS dea
JOIN CovidVaccinations AS vac
    ON dea.location = vac.location
   AND dea.date = vac.date
WHERE dea.continent IS NOT NULL;

In [ ]:
%%sql
SELECT name
FROM sqlite_master
WHERE type='view';
